## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Weather_Database/WeatherPy_database.csv'

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
10,10,Puerto Ayora,EC,2021-10-31 01:37:03,-0.7393,-90.3518,70.65,85,100,7.27
13,13,Alta Floresta,BR,2021-10-31 01:37:04,-9.8756,-56.0861,74.61,79,86,3.33
14,14,Ribeira Grande,PT,2021-10-31 01:37:04,38.5167,-28.7000,70.86,88,100,18.19
16,16,Rikitea,PF,2021-10-31 01:37:05,-23.1203,-134.9692,74.59,87,100,21.83
39,39,Arraial Do Cabo,BR,2021-10-31 01:35:57,-22.9661,-42.0278,70.68,88,100,17.29
44,44,Soyo,AO,2021-10-31 01:37:17,-6.1349,12.3689,77.14,85,100,9.33
56,56,Veraval,IN,2021-10-31 01:38:23,20.9000,70.3667,74.79,45,84,11.07
57,57,Grand Gaube,MU,2021-10-31 01:38:23,-20.0064,57.6608,70.38,84,0,3.00
61,61,Carnarvon,AU,2021-10-31 01:38:25,-24.8667,113.6333,73.47,60,0,17.27
62,62,Payao,PH,2021-10-31 01:38:26,10.1833,122.9500,79.57,85,100,10.60


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID       171
City          171
Country       171
Date          171
Lat           171
Lng           171
Max Temp      171
Humidity      171
Cloudiness    171
Wind Speed    171
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
10,Puerto Ayora,EC,70.65,-0.7393,-90.3518,
13,Alta Floresta,BR,74.61,-9.8756,-56.0861,
14,Ribeira Grande,PT,70.86,38.5167,-28.7000,
16,Rikitea,PF,74.59,-23.1203,-134.9692,
39,Arraial Do Cabo,BR,70.68,-22.9661,-42.0278,
44,Soyo,AO,77.14,-6.1349,12.3689,
56,Veraval,IN,74.79,20.9000,70.3667,
57,Grand Gaube,MU,70.38,-20.0064,57.6608,
61,Carnarvon,AU,73.47,-24.8667,113.6333,
62,Payao,PH,79.57,10.1833,122.9500,


In [11]:
gmaps.configure(api_key=g_key)
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df

In [15]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_vacation.csv"
output_data_file = "Weather_Database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))